<a href="https://colab.research.google.com/github/inderpreetsingh01/Speech/blob/main/computing_wer_for_custom_data_on_huggingface_wav2vec2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!apt-get install libsox-fmt-all libsox-dev sox

In [3]:
%%capture
!pip install pytube pydub transformers datasets jiwer sox

In [4]:
import sox

In [7]:
import librosa

In [9]:
import os
import re
import sox
import librosa
from pytube import YouTube
from pydub import AudioSegment
# from pydub.utils import make_chunks #may not need going forward
from IPython.display import Audio, display

In [10]:
import pandas as pd

In [11]:
def retrieve_audio(video_link, output_dir):

    ROOT = os.getcwd()
    output = os.path.join(ROOT, output_dir)
    os.makedirs(output, exist_ok=True)
    
    try:
        video = YouTube(video_link)

        audio = video.streams.filter(only_audio=True, file_extension='mp4').first()

        audio.download(output_dir)
    
    except:
        print("Connection error")

In [12]:
#add denoising here as it becomes available

def convert_to_wav(input_path, output_dir):
    
    ROOT = os.getcwd()
    output = os.path.join(ROOT, output_dir)
    os.makedirs(output, exist_ok=True)

    file_name = re.split("/|\.", input_path)[-2]

    audio = AudioSegment.from_file(input_path)
    audio.export(f"{output_dir}/{file_name}.wav", format='wav')

In [12]:
retrieve_audio("https://www.youtube.com/watch?v=jlJijwgtgzI", "tmp")
convert_to_wav('tmp/A Panel Discussion on Climate Change in New England.mp4', "input")
# tfm = sox.Transformer()
# tfm.trim(0, 600)
# tfm.build_file("input/A Panel Discussion on Climate Change in New England.wav", "input/A Panel Discussion on Climate Change in New England.wav")
DEMO_FILE = {'name': "A Panel Discussion on Climate Change in New England", 'audio': "input/A Panel Discussion on Climate Change in New England.wav"}

In [13]:
SAMPLE_RATE=16000
audio, sr = librosa.load('input/A Panel Discussion on Climate Change in New England.wav', sr=SAMPLE_RATE)

display(Audio(audio, rate=sr))

In [16]:
repo_name = 'facebook/wav2vec2-large-960h-lv60-self'

In [17]:
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC
processor = Wav2Vec2Processor.from_pretrained(repo_name)
model = Wav2Vec2ForCTC.from_pretrained(repo_name)

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-large-960h-lv60-self and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
tfm = sox.Transformer()
tfm.trim(0, 60)
tfm.build_file("input/A Panel Discussion on Climate Change in New England.wav", "input/A Panel Discussion on Climate Change in New England trimmed 1min.wav")

True

In [13]:
SAMPLE_RATE=16000
audio_trimmed, sr = librosa.load('input/A Panel Discussion on Climate Change in New England trimmed 1min.wav', sr=SAMPLE_RATE)

display(Audio(audio_trimmed, rate=sr))

In [19]:
import torch
with torch.no_grad():
  input_values = processor(audio_trimmed, return_tensors="pt", spadding="longest", sampling_rate=SAMPLE_RATE).input_values
  logits = model(input_values).logits
  predicted_ids = torch.argmax(logits, dim=-1)
  pred_transcript = processor.batch_decode(predicted_ids)

In [20]:
from datasets import load_metric

In [21]:
wer_metric = load_metric("wer")

In [22]:
with open('/content/my_transcript_text_only.txt', 'r') as f:
  ref_text = f.read()

In [23]:
ref_text

"so it's been interesting look I'm coming in and it looks like into two major groups and the first being this question of adaptation how do we respond right here in New England to these kinds of impacts the other main theme that's very clear is people wanting to move very quickly beyond this question of adaptation to mitigation given the scale and size of the impacts and the warming that we're now expecting some many people in the audience are actually quite interested in moving on to well what do we do to prevent these changes to begin with but let's start with the adaptation side and there's a number of comments here that are stronger than the way I would say this so but let me say it this way that's nuts that's why we have questions from the audience my question would be the given the sea level rise and greater storm surge is now expected in the future how should we be rethinking coastal zoning policies other people put it a little more bluntly shouldn't we ban zone or zone out all 

In [24]:
print("Test WER: {:.3f}".format(wer_metric.compute(predictions=pred_transcript, references=[ref_text[:780]])))

Test WER: 1.055
